# Preprocessing of data for:

1. Load required data and provide some basic stats
2. Remove all articles that have been wrongly aggregated by Facebook
3. Process collected metrics
4. Match articles with disciplinary information
5. Write output files used in analysis notebook

In [97]:
import pandas as pd

## 1. Load required data and provide some basic stats

In [98]:
# Input data
disciplines_csv = "data/external/PLOS_2015-2017_idArt-DOI-PY-Journal-Title-LargerDiscipline-Discipline-Specialty.csv"

in_articles_csv = "data/input/plos_one_articles.csv"
details_csv = "data/input/query_details.csv"
fb_metrics_csv = "data/input/graph_api_counts.csv"

# Output data
out_articles_csv = "data/articles.csv"
out_responses_csv = "data/responses.csv"

In [ ]:
# Load articles and extract years
all_articles = pd.read_csv(in_articles_csv, index_col="doi", parse_dates=['publication_date'])
all_articles['year'] = all_articles.publication_date.map(lambda x: x.year)

# Replace authors of articles by PLOS without author information with "PLOS ONE"
all_articles.loc[all_articles[all_articles.author.isna()].index, "author"] = "PLOS ONE"

In [109]:
# Load responses
all_responses = pd.read_csv(details_csv, index_col="id", parse_dates=['received_at', 'og_updated_time', 'publication_date', 'added_on'])

In [109]:
# Load both metrics files and merge
all_metrics = pd.read_csv(fb_metrics_csv, index_col="doi")

# Rename the metrics
col_names = {
    'shares': 'AES_og',
    'reactions': 'AER_og',
    'comments' : 'AEC_og'
}
all_metrics.rename(columns=col_names, inplace=True)

In [109]:
# Load disciplines
disciplines = pd.read_csv(disciplines_csv, delimiter=";", index_col="DOI")
disciplines.index = disciplines.index.map(lambda x: str(x)[4:])

# Rename columns
col_names = {
    "EGrande_Discipline": "grand_discipline",
    "EDiscipline": "discipline",
    "ESpecialite": "specialty"
}
disciplines.rename(columns=col_names, inplace=True)

**Some basic stats about the data before processing steps were applied:**

In [101]:
print("FB responses - responses:", fb_metrics.shape[0])
print("FB responses - non-zero responses", fb_metrics.dropna(how="all").shape[0])
print("FB responses with at least one share:", fb_metrics.shares.replace(0, np.nan).count())
print("FB responses with at least one reaction:", fb_metrics.reactions.replace(0, np.nan).count())
print("FB responses with at least one comment:", fb_metrics.comments.replace(0, np.nan).count())
print("FB responses with at least one plugin comment:", fb_metrics.plugin_comments.replace(0, np.nan).count())

FB responses - responses: 49121
FB responses - non-zero responses 49121
FB responses with at least one share: 21439
FB responses with at least one reaction: 13849
FB responses with at least one comment: 10283
FB responses with at least one plugin comment: 0


In [102]:
n = len(all_metrics.replace(0, np.nan)[['AES_og', 'AER_og', 'AEC_og']].dropna(how="all"))
print("Articles with at least one share, reaction, or comment on Facebook: {}".format(n))

Articles with at least one share, reaction, or comment on Facebook: 21698


In [103]:
n_responses = all_responses[['reactions', 'shares', 'comments']].shape[0]
print("FB queries that returned results: {} ({:.2f}%)".format(n_responses, 100 * n_responses / all_articles.shape[0] / 10))

zero_eng = sum(all_responses[['reactions', 'shares', 'comments']].sum(axis=1)==0)
print("Responses with no engagement at all: {} ({:.2f}%)".format(zero_eng, 100*zero_eng/n_responses))

FB queries that returned results: 69983 (11.31%)
Responses with no engagement at all: 37062 (52.96%)


In [104]:
print("FB queries with results:", all_responses.shape[0])
print("Found articles:", all_articles.shape[0])
print("Found articles with metrics:", all_metrics.shape[0])

FB queries with results: 69983
Found articles: 61872
Found articles with metrics: 61872


## 2. Remove all articles that have been wrongly aggregated by Facebook

The following steps removes articles that were wrongly aggregated within the Facebook social graph. See Enkhbayar and Alperin (2018) for more information.

In [105]:
ogid_counts = all_responses.groupby(["doi", "og_id"]).size().groupby(['og_id']).count()

bad_ogids = ogid_counts[ogid_counts>1].keys()
bad_dois = all_responses[all_responses.og_id.isin(bad_ogids)].doi

responses = all_responses[~all_responses.doi.isin(bad_dois)]
articles = all_articles.drop(bad_dois, axis=0)
metrics = all_metrics.drop(bad_dois, axis=0)

dropped_years = all_articles.reindex(bad_dois).year.value_counts()

# Article counts - base dataset
n_articles_by_year = articles.groupby("year").count()['title']
n_all_articles_by_year = all_articles.groupby("year").count()['title']

df_article_counts = pd.DataFrame({
    'All articles': n_all_articles_by_year,
    'Dropped': dropped_years,
    'Final article count': n_articles_by_year
})
df_article_counts.loc['All years'] = df_article_counts.sum(axis=0)
df_article_counts.index.name = ""

df_article_counts

,All articles,Dropped,Final article count
,,,
2015,25436,9,25427
2016,18815,6,18809
2017,17621,9,17612
All years,61872,24,61848


## 3. Process collected metrics

Replace zero counts with NAs.

In [107]:
# Replace any fb metric of 0 with nan
for _ in ['AES', 'AER', 'AEC']:
    all_metrics[_] = all_metrics[_+"_og"][all_metrics[_+"_og"] != 0]
    
x = all_metrics[['AES_og', 'AEC_og', 'AER_og']] == 0
print("{} articles with 0/0/0 AE".format(x.all(axis=1).sum()))

27423 articles with 0/0/0 AE


In [108]:
# Add metrics to articles
articles = articles.join(all_metrics[["AES", "AER", "AEC"]])

## 4. Match articles with disciplinary information

Disciplinary information for each article is provided by Piwowar et al. (2018). In order to use the data the articles were matched with several steps:

1. Match articles by DOIs
2. Match articles by titles (after conversion to alphanum & lowercase)

In 6 cases the disciplinary information provided multiple disciplines for articles in which we chose one randomly.

In [110]:
# Convert titles to alphanum & lowercase for in both datasets
articles['title_'] = articles['title'].map(lambda x: ''.join(e for e in x.lower() if e.isalnum()))
disciplines['title_'] = disciplines['title'].map(lambda x: ''.join(e for e in x.lower() if e.isalnum()))

In [111]:
# Naive join of articles and disciplinary information by DOIs
x = articles.join(disciplines[["grand_discipline", "discipline", "specialty"]], how="left")

# Articles with multiple disciplines
print(x.index.duplicated().sum(), "articles with multiple disciplines. Selecting one randomly.")

# Dropping of the duplicate disciplines randomly
x = x[~x.index.duplicated()]

6 articles with multiple disciplines. Selecting one randomly.


In [113]:
# select those that still miss disciplines
missings = x[x.discipline.isna()].copy()
missings = missings.drop(["grand_discipline", "discipline", "specialty"], axis=1)
print("Missing articles after DOI matching:", missings.shape[0])

Missing articles after DOI matching: 7989


In [115]:
# try to match these with titles and replace in x
found = missings.reset_index().merge(disciplines[["grand_discipline", "discipline", "specialty", "title_"]], left_on="title_", right_on="title_", how="inner").set_index('index')
print("Title matching found:", found.shape[0])
x.loc[found.index] = found

# select those that still miss disciplines
missings = x[x.discipline.isna()].copy()
missings = missings.drop(["grand_discipline", "discipline", "specialty"], axis=1)
print("Missing articles after title matching:", missings.shape[0])

Title matching found: 4060
Missing articles after title matching: 3929


In [55]:
# drop temp column with modified titles
articles = x.drop(columns="title_")

In [122]:
author_plos = articles.author.str.contains("PLOS")
type_corr = articles.title.str.contains("Correction: ")
type_retr = articles.title.str.contains("Retraction: ")

print("Artices by PLOS", author_plos.sum())
print("Corrections", type_corr.sum())
print("Retractions", type_retr.sum())

Artices by PLOS 1197
Corrections 3332
Retractions 36


In [131]:
df = pd.DataFrame(columns=["Count"])
df.loc['Articles with disciplines'] = sum(~x.discipline.isna())
df.loc['Articles not in disc. dataset'] = sum([any(x) for x in zip(author_plos, type_corr, type_retr)])
df.loc['Actual missing articles'] = x.discipline.isna().sum()-sum([any(x) for x in zip(author_plos, type_corr, type_retr)])
df.loc['Sum'] = df.sum()
df

,Count
Articles with disciplines,57919
Articles not in disc. dataset,3374
Actual missing articles,555
Sum,61848


## 5. Write output files used in analysis notebook

In [58]:
articles.index.name = "doi"

articles.to_csv("data/articles.csv")
responses.to_csv("data/responses.csv")

# References

Enkhbayar, A., & Alperin, J. P. (2018). Challenges of capturing engagement on Facebook for Altmetrics. STI 2018 Conference Proceedings, 1460–1469. Retrieved from http://arxiv.org/abs/1809.01194

Piwowar, H., Priem, J., Larivière, V., Alperin, J. P., Matthias, L., Norlander, B., … Haustein, S. (2018). The state of OA: A large-scale analysis of the prevalence and impact of Open Access articles. PeerJ, 6, e4375. doi: [10/ckh5](https://doi.org/10/ckh5)